## Covid-19 Geospatial Data Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
from shapely.geometry import Point, Polygon

### Import Data

In [ ]:
#location of Johns Hopkins University Data: https://github.com/CSSEGISandData/COVID-19

#Johns Hopkins Data on Cases and Deaths
cases_us_full = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths_us_full = pd.read_csv('time_series_covid19_deaths_US.csv')

#Data on population of each US county
pop_us = pd.read_csv('DECENNIALSF12010.P1_data_with_overlays_2021-12-03T213906.csv')

#CDC data on current vaccinations per US county
vac_us_full = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_County.csv')

#County areas in sq miles
land_full=pd.read_excel('LND01.xls')


### Data Cleaning

In [ ]:
cases_us_full.head(5)

In [ ]:
x=cases_us_full.columns.to_list()

In [ ]:
#create new df of cases with only selected columns
cases_us = cases_us_full[[x[0],x[5],x[6],x[7],x[8],x[9],x[-1]]].copy()

#strip down UID column to get standard county ID for joining data from other df.
cases_us['ID'] = cases_us['UID'].astype(str).str[-5:]
#cases_us['ID'] = cases_us['ID'].astype(int)

In [ ]:
cases_us = cases_us.rename(columns={'1/21/22':'Cases'})

In [ ]:
df_us = cases_us.copy()

In [ ]:
df_us['Deaths'] = deaths_us_full['1/21/22']

In [ ]:
pop_us.head()

In [ ]:
pop_us=pop_us.drop([0])

In [ ]:
pop_us=pop_us.rename(columns={'P001001':'Population'})

#strip down GEO_ID column to get standard county ID for joining data from other df.
pop_us['pop_ID']=pop_us['GEO_ID'].str[-5:]
#pop_us['pop_ID'] = pop_us['pop_ID'].astype(int)

In [ ]:
#cleaning to remove the xxxx(rxxx) in the population column
pop_us['Population'].astype(str)
pattern = '\(......\)'
pop_us['Population']=pop_us.Population.str.replace(pat=pattern,repl= "", regex=True)
pop_us['Population'] = pop_us['Population'].astype(int)
#pop_us['pop_ID'] = pop_us['pop_ID'].astype(int)

In [ ]:
#merge in population for each county to overall df
df_us=df_us.merge(pop_us[['Population','pop_ID']], how='left', left_on='ID', right_on='pop_ID')

In [ ]:
vac_us_full = vac_us_full.rename(columns={'Administered_Dose1_Pop_Pct':'Vaccinated_Percent'})

In [ ]:
#merge in vaccinated percent for each county to overall df
df_us = df_us.merge(vac_us_full, how='left', left_on='ID', right_on='FIPS')

In [ ]:
df_us['Population'] = df_us['Population'].astype(float)

In [ ]:
# create feature for cases as percent of county population
df_us['Cases_Percent']=((df_us['Cases']) / (df_us['Population']))*100

In [ ]:
# create feature for deaths as percent of county population
df_us['Deaths_Percent']=((df_us['Deaths']) / (df_us['Population']))*100

In [ ]:
df_us.shape

In [ ]:
df_us = df_us.dropna()
df_us.shape

In [ ]:
land_full.head()

In [ ]:
#new df with only selected columns
land=land_full[['Areaname', 'STCOU', 'LND110210D']]
land.head()
#LND in sq miles

In [ ]:
land['STCOU'] = land['STCOU'].astype(float)
df_us['ID'] = df_us['ID'].astype(float)

In [ ]:
#merge the land areas per county into the overall df 
df_us=df_us.merge(land, how='left', left_on='ID', right_on='STCOU')

In [ ]:
#create feature for ppl/sq mi density
df_us['Pop_Density'] = (df_us['Population'])/(df_us['LND110210D'])

In [ ]:
df_us.describe()

In [ ]:
pd.options.display.max_rows=250
df_us.sort_values(by='Pop_Density', ascending=False)

#Census Bureau defines divide between urban/rural as 1,000 ppl/sq mile

In [ ]:
#remove missing values for Puerto Rico
df_us=df_us.dropna(thresh=16)
df_us.shape

In [ ]:
#Number of counties with 0 cases. Most likely these counties arent reporting. But cant exclude because we cant 
# say that for sure
df_us[df_us['Cases']==0].count()['Cases']

In [ ]:
#Number of counties with 0 deaths. Most likely these counties arent reporting. But cant exclude because we cant 
# say that for sure
df_us[df_us['Deaths']==0].count()['Deaths']

### Exploratory Data Analysis / Descriptive Statistics

In [ ]:
df_us.describe()

In [ ]:
#Figure 1
sns.histplot(data=df_us, x='Cases_Percent', bins=50)

In [ ]:
#Figure 2
sns.histplot(data=df_us, x='Deaths_Percent', bins=50)

In [ ]:
#Figure 3
sns.histplot(data=df_us, x='Vaccinated_Percent', bins=50)

In [ ]:
#Figure 4
sns.histplot(data=df_us, x='Pop_Density', bins='auto', log_scale=True)
#log scale

Cases and Death Percentages were very normally distributed aside from counties that are believed to not be reporting. The Vaccinated Percentage was not as normally distributed as compared to Cases and Death Percentage histograms. 

Is there a difference in Cases and Deaths Percentages based upon population density?
3 different population densities will be used to determine any differences. 
1) > 1000 people/sq mi

2) > 100 & < 1000 people/sq mi

3) < 100 people/sq mi

In [ ]:
#Figure 5


#break the pop density into 3 cohorts based upon population densities, and see if there are more differences

fig, axes = plt.subplots(3,3, figsize=(15,15), sharey=True)


Filt_urban2 = df_us[df_us['Pop_Density'] > 1000]
Filt_middle2 = df_us[(df_us['Pop_Density'] > 100) & (df_us['Pop_Density'] <= 1000)]
Filt_rural2 = df_us[df_us['Pop_Density'] <= 100]

sns.histplot(ax=axes[0,0], data=Filt_urban2, x='Cases_Percent', bins='auto')
sns.histplot(ax=axes[0,1], data=Filt_middle2, x='Cases_Percent', bins='auto')
sns.histplot(ax=axes[0,2], data=Filt_rural2, x='Cases_Percent', bins='auto')

sns.histplot(ax=axes[1,0], data=Filt_urban2, x='Deaths_Percent', bins='auto')
sns.histplot(ax=axes[1,1], data=Filt_middle2, x='Deaths_Percent', bins='auto')
sns.histplot(ax=axes[1,2], data=Filt_rural2, x='Deaths_Percent', bins='auto')

sns.histplot(ax=axes[2,0], data=Filt_urban2, x='Vaccinated_Percent', bins='auto')
sns.histplot(ax=axes[2,1], data=Filt_middle2, x='Vaccinated_Percent', bins='auto')
sns.histplot(ax=axes[2,2], data=Filt_rural2, x='Vaccinated_Percent', bins='auto')

plt.show()

In [ ]:
#Table 1
Filt_urban2.describe()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']]

In [ ]:
#Table 2
Filt_middle2.describe()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']]

In [ ]:
#Table 3
Filt_rural2.describe()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']]

Result: All 3 cohorts showed very normally distributed data in Cases Percent and Deaths Percent. Cases Percent median for all 3 cohorts was within +/- 1% of 21.5% with no pattern based upon population density. Deaths Percent median for a 3 cohorts was within +/- 2.5% of 28.5%, and the Death Percent increased as the population density decreased. 

Data for Vaccinated Percent was normally distributed but retained some skew for the most urban cohort and most rural cohort. A drill down into the highly vaccinated urban counties (upper 25% quartile) and lowest vaccinated rural counties (34% and lower) will be performed for further insights.

In [ ]:
Filt_urban3 = Filt_urban2[Filt_urban2['Vaccinated_Percent']>86.725]

In [ ]:
Filt_rural3 = Filt_rural2[Filt_rural2['Vaccinated_Percent']<34.00]

In [ ]:
#Figure 6

fig, axes = plt.subplots(3,2, figsize=(15,15), sharey=True)

sns.histplot(ax=axes[0,0], data=Filt_urban3, x='Cases_Percent', bins='auto')
sns.histplot(ax=axes[0,1], data=Filt_rural3, x='Cases_Percent', bins='auto')

sns.histplot(ax=axes[1,0], data=Filt_urban3, x='Deaths_Percent', bins='auto')
sns.histplot(ax=axes[1,1], data=Filt_rural3, x='Deaths_Percent', bins='auto')

sns.histplot(ax=axes[2,0], data=Filt_urban3, x='Vaccinated_Percent', bins='auto')
sns.histplot(ax=axes[2,1], data=Filt_rural3, x='Vaccinated_Percent', bins='auto')

plt.show()

In [ ]:
#Table 4
Filt_urban3.describe()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']]

In [ ]:
#Table 5
Filt_rural3.describe()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']]

Results: Table 4 shows the upper 25% quartile of Vaccinated Percent of urban areas had a slightly higher median Deaths Percent but a much lower Cases Percent when compared to the overall urban areas population density. 

Table 5 shows the lowest vaccinated rural areas, a median rate of 23.9% vaccinated. The Cases Percent and Deaths Percent median were 16.5% and 0.17%, respectively. The Deaths Percent data was skewed, and the mean was 0.21%. Surprisingly, this lowely vaccinated area has a lower Death Percent than even the most highly vaccinated urban area. Another factor to consider is Population Density associated with Table 5. The median population density for this group is 7 people per square mile. The general lack of humans to pass on the infection should be a consideration in this analysis.

## Correlation and Cluster Analysis

### What are the correlations, if any, between vaccination, cases, deaths, and population density?

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
df_us.head()

In [ ]:
df_clust = df_us[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']]
df_clust

In [ ]:
scaler = StandardScaler()
df_stand = scaler.fit_transform(df_clust)

In [ ]:
wcss=[]
for i in range(1,10):
    kmeans = KMeans(n_clusters=i, init = 'k-means++', random_state = 20)
    kmeans.fit(df_stand)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,10),wcss)
plt.show()

In [ ]:
#select number of clusters as 5
kmeans = KMeans(n_clusters=5, init = 'k-means++', random_state = 20)
y_kmeans = kmeans.fit_predict(df_stand)

In [ ]:
df_clust['cluster_number'] = y_kmeans
df_us['cluster_number'] = y_kmeans

In [ ]:
#Table 6
#show how many data points assigned to each cluster
df_clust.groupby(by='cluster_number').count()

In [ ]:
#Figure 7

plt.figure(figsize=(10,10))
sns.heatmap(data=df_clust.corr(), annot=True)

In [ ]:
# Figure 8
sns.pairplot(data=df_clust, hue='cluster_number', palette='deep')

In [ ]:
#Figure 9

fig, axes = plt.subplots(4, figsize=(15,15), sharey=True)

sns.scatterplot(ax=axes[0], data=df_us, hue='cluster_number', x='Cases_Percent', y='Vaccinated_Percent', palette='deep')
sns.scatterplot(ax=axes[1], data=df_us, hue='cluster_number', x='Deaths_Percent', y='Vaccinated_Percent', palette='deep')
axes[2].set(xscale='log')
sns.scatterplot(ax=axes[2], data=df_us, hue='cluster_number', x='Pop_Density', y='Vaccinated_Percent', palette='deep')
sns.scatterplot(ax=axes[3], data=df_us, hue='cluster_number', x='Deaths_Percent', y='Cases_Percent', palette='deep')

In [ ]:
#Table 7

for i in range(0,5):
    print(df_clust[df_clust['cluster_number']==i].describe()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']])

In [ ]:
#Table 8

for i in range(0,5):
    print(df_clust[df_clust['cluster_number']==i].median()[['Vaccinated_Percent', 'Cases_Percent', 'Deaths_Percent', 'Pop_Density']])

### Results: 

Cluster Number had the strongest correlation to Cases Percent as shown in Figure 7. Therefore, Cases Percent was the most important factor in determining Cluster Number. Population Density had almost no correlation to anything except Vaccinated Percent. It was surprising that Population Density did not correlate to Cases Percent because it was expected a high population density would make it harder to not be amongst other people, and thus more likely to become infected. Aside from cluster number, the next strongest correlation was Cases Percent vs Death Percent which was an expected correlation.

Cluster 3 contained only 3 points as shown in Table 6. Those 3 points were also the largest population densities. This is too small of sample size to draw conclusions.

The Cases Percent and Deaths Percent standard deviation amongst all the clusters was very similar. The Vaccination Percent standard deviation amonst all the clusters was not as consistant as shown in Table 7.

The clusters can be approximately described as such from the clusters' medians in Table 8:

Cluster 0: high vaccination, low cases, low deaths, mid to high population density, 629 data points

Cluster 1: low vaccination, mid cases, high deaths, low to mid population density, 916 data points 

Cluster 2: low vaccination, low cases, low deaths, low population density, 546 data points

Cluster 3: high vaccination, high cases, high deaths, high population density, 3 data points

Cluster 4: mid vaccination, high cases, mid deaths, mid population density, 1046 data points 

## Geospatial Analysis

### How do the cluster location look when plotted on a map? Are there any additional correlations that can be found?

In [ ]:
df_us.head()

In [ ]:
#read in US county shape file
county_shp = gpd.read_file('.\cb_2018_us_county_500k\cb_2018_us_county_500k.shp')

In [ ]:
# merge county shape geometry into df_us to make required geometry for geodataframe 
df_us = df_us.merge(county_shp[['geometry', 'GEOID']], how='left', left_on='pop_ID', right_on='GEOID')

In [ ]:
#create geodataframe
geo_df = gpd.GeoDataFrame(df_us, crs=4326, geometry=df_us['geometry'])

In [ ]:
# Figure 10

from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1,1, figsize=(20,20))

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.1)
ax.set_xlim(-130,-60)
ax.set_ylim(25,50)
plt.title('County Fatalities (Percent)', loc='center')

geo_df.plot(ax=ax, column='Deaths_Percent', legend=True, cax=cax)

plt.show()

In [ ]:
# Figure 11

geo_df.explore(legend=True, column='Vaccinated_Percent', tooltip=['Cases_Percent', 'Cases', 'Deaths_Percent', 'Deaths', 'Vaccinated_Percent', 'Areaname', 'Population'])

In [ ]:
# Figure 12

m = geo_df[geo_df['cluster_number']==0].explore(color='blue', tooltip=['cluster_number', 'Cases_Percent', 'Cases', 'Deaths_Percent', 'Deaths', 'Vaccinated_Percent', 'Areaname', 'Population'])
geo_df[geo_df['cluster_number']==1].explore(m=m, color='orange', tooltip=['cluster_number', 'Cases_Percent', 'Cases', 'Deaths_Percent', 'Deaths', 'Vaccinated_Percent', 'Areaname', 'Population'])
geo_df[geo_df['cluster_number']==2].explore(m=m, color='green', tooltip=['cluster_number', 'Cases_Percent', 'Cases', 'Deaths_Percent', 'Deaths', 'Vaccinated_Percent', 'Areaname', 'Population'])
geo_df[geo_df['cluster_number']==3].explore(m=m, color='red', tooltip=['cluster_number', 'Cases_Percent', 'Cases', 'Deaths_Percent', 'Deaths', 'Vaccinated_Percent', 'Areaname', 'Population'])
geo_df[geo_df['cluster_number']==4].explore(m=m, color='purple', tooltip=['cluster_number', 'Cases_Percent', 'Cases', 'Deaths_Percent', 'Deaths', 'Vaccinated_Percent', 'Areaname', 'Population'])

m

### Results:

Cluster 3 (red) all 3 data points are located in New York City.

Cluster 2 (green) had the lowest median population density. That makes sense when the majority of its data points are in the west.

Cluster 0 (blue) is predominantly found on the west coast or the northeast.

Clusters 1 (orange) and 4 (purple) are predominantly in the south and midwest.

In [ ]:
#https://covid.cdc.gov/covid-data-tracker/#county-view?list_select_state=all_states&list_select_county=all_counties&data-type=Vaccinations&metric=Administered_Dose1_Pop_Pct
# https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh/data
